In [ ]:
%matplotlib inline
import librosa, librosa.display
import IPython.display as ipd
import vamp
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft

In [ ]:
import re

In [ ]:
import vamp

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint

In [ ]:
beatles_live = librosa.load('/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Beatles - I wanna hold your hand - live.m4a')
beatles = librosa.load('/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Beatles - I wanna hold your hand -studio.m4a')

In [ ]:
cow = librosa.load('/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Kuh muht-TdheW61w4Co.m4a')

In [ ]:
cow_chords = vamp.collect(cow[0], cow[1], "nnls-chroma:chordino")['list']

In [ ]:
pprint(list(enumerate(cow_chords)))

In [ ]:
label = 'N'
ind = 0
while label == 'N':
    ind += 1
    label = cow_chords[ind]['label']
beginning_ind = ind
print(beginning_ind)
while label != 'N':
    ind += 1
    label = cow_chords[ind]['label']
end_ind = ind - 1
print(end_ind)

timestamps_cow = [float(cow_chords[beginning_ind]['timestamp']), float(cow_chords[2]['timestamp'])]
bounds_cow = [librosa.time_to_samples(t, cow[1]) for t in timestamps_cow]
print(bounds_cow)
bounds_cow[0]=bounds_cow[0]-25000
bounds_cow[1]=bounds_cow[0]+35000
cow_in_F = cow[0][bounds_cow[0]:bounds_cow[1]]
ipd.Audio(cow_in_F, rate=cow[1])

In [ ]:
ipd.Audio(cow_in_F, rate=cow[1]*2**(1/12))

In [ ]:
def last_not_0_before_par(par):
    ind = par
    while cow[0][ind] == 0:
        ind -= 1
    return ind

def first_0_before_par(par):
    ind = par
    while cow[0][ind] != 0:
        ind -= 1
    return ind

def sign_change(par):
    ind = par
    while cow[0][ind] * cow[0][par] > 0:
        ind -= 1
    return ind

In [ ]:
type(cow[0])

In [ ]:
last_int_sample = last_not_0_before_par(bounds_cow[1])
sample = last_int_sample
for _  in range(50):
    sample = sign_change(sample-1)
last_sample_extension = sample

for _  in range(1000):
    sample = sign_change(sample-1)
first_sample_extension = sample
    
#first_sample_extension = sample
print(last_int_sample)
print(first_sample_extension)

long_sample_cow = cow[0][bounds_cow[0]:bounds_cow[1]]
for _ in range(10):
    long_sample_cow = np.concatenate((long_sample_cow, cow[0][first_sample_extension:last_sample_extension]))
print(len(long_sample_cow))
ipd.Audio(long_sample_cow, rate=cow[1])

In [ ]:
def rate_cow(num_seminotes_above_F_sharp):
    return cow[1]*2**num_seminotes_above_F_sharp

In [ ]:
def long_enough_cow(num_samples, rate_song, rate_cow):
    output = cow[0][bounds_cow[0]:bounds_cow[1]]
    while len(output) < num_samples*rate_cow/rate_song:
        output = np.concatenate((output, cow[0][first_sample_extension:last_sample_extension]))
    return output

In [ ]:
def long_enough_ignoring_frequencies(num_samples):
    output = cow[0][bounds_cow[0]:bounds_cow[1]]
    while len(output) < num_samples:
        output = np.concatenate((output, cow[0][first_sample_extension:last_sample_extension]))
    return output

In [ ]:
add_cow = np.zeros(len(beatles[0]))
max_num_samples = max([end-beginning for beginning, end in bounds_G])
long_cow = long_enough_ignoring_frequencies(max_num_samples)
for beginning, end in bounds_G:
    if end > len(beatles[0]):
        end = len(beatles[0])
    add = long_cow[:(end-beginning)]
    add_cow[beginning:end] = add
print(add_cow_G)
print(len(add_cow_G))
print(len(beatles[0]))
output = beatles[0] + add_cow
ipd.Audio(output, rate=beatles[1])

In [ ]:
def function_cow(num_samples, semitones_above_F_sharp):
    cow_rate = rate_cow(semitones_above_F_sharp)
    long_cow = long_enough_cow(num_samples, beatles[1], semitones_above_F_sharp)
    cow_list = [long_cow[0+cow_rate//beatles[1]]*i for i in range(num_samples)]
    output = np.array(cow_list, np.float32)
    return output

In [ ]:
cow_rate = rate_cow(1)
print(cow_rate//beatles[1])
beginning, end = bounds_G[0]
add = function_cow(end-beginning, 1)
print(add[0:50])
ipd.Audio(add, rate=beatles[1])

In [ ]:
add_cow_G = np.zeros(len(beatles[0]))
for beginning, end in bounds_G:
    if end > len(beatles[0]):
        end = len(beatles[0])
    add = function_cow(end-beginning, 1)
    add_cow_G[beginning:end] = add
print(add_cow_G)
print(len(add_cow_G))
print(len(beatles[0]))
ipd.Audio(add_cow_G, rate=beatles[1])
#output = beatles[0] + add_cow_G
#ipd.Audio(output, rate=beatles[1])

In [ ]:
beatles[0].shape

In [ ]:
#beatles[0][100:1000]

In [ ]:
ipd.Audio(beatles[0], rate=beatles[1])

In [ ]:
def note_by_seconds(freq, len, amp=1, rate=beatles[1]):
    t = np.linspace(0,len,len*rate)
    data = np.sin(2*np.pi*freq*t)*amp
    return data

In [ ]:
A_2sec = note_by_seconds(220, 4)
print(type(A_2sec))
ipd.Audio(A_2sec, rate=beatles[1])

In [ ]:
def note_by_num_samples(freq, num_samples, amp=1, rate=beatles[1]):
    data = np.array([np.sin(2*np.pi*freq*librosa.samples_to_time(sample, rate))*amp for sample in range(num_samples)], dtype=np.float32)
    return data

In [ ]:
A_10000samples = note_by_num_samples(220, 50000)
print(type(A_10000samples))
ipd.Audio(A_10000samples, rate=beatles[1])

In [ ]:
librosa.display.waveplot(beatles[0])

In [ ]:
ipd.Audio(beatles[0], rate = beatles[1])

In [ ]:
vamp.list_plugins()
help(vamp.list_plugins)

In [ ]:
chords = vamp.collect(beatles[0], beatles[1], "nnls-chroma:chordino")

In [ ]:
pprint(list(enumerate(chords['list'])))

In [ ]:
chords['list'][22]['label'] == 'Ebm'

In [ ]:
chord_pairs = list(zip(chords['list'][:-1], chords['list'][1:]))
timestamps = [[float(c[0]['timestamp']), float(c[1]['timestamp'])] for c in chord_pairs]
bounds = [librosa.time_to_samples(t, beatles[1]) for t in timestamps]

chord_number = 2
print(chords['list'][chord_number]['label'], bounds[chord_number])
ipd.Audio(beatles[0][bounds[chord_number][0]:bounds[chord_number][1]], rate=beatles[1])

In [ ]:
timestamps_G = [ (float(c[0]['timestamp']), float(c[1]['timestamp'])) for c in chord_pairs if re.match(r'G.*', c[0]['label'])]
bounds_G = [librosa.time_to_samples(t, beatles[1]) for t in timestamps_G]
lables_G = [c[0]['label'] for c in chord_pairs if re.match(r'G.*', c[0]['label'])]

chord_number = 2
print(lables_G[chord_number])
ipd.Audio(beatles[0][bounds_G[chord_number][0]:bounds_G[chord_number][1]], rate=beatles[1])

In [ ]:
timestamps_D = [ (float(c[0]['timestamp']), float(c[1]['timestamp'])) for c in chord_pairs if re.match(r'D.*', c[0]['label'])]
bounds_D = [librosa.time_to_samples(t, beatles[1]) for t in timestamps_D]
lables_D = [c[0]['label'] for c in chord_pairs if re.match(r'D.*', c[0]['label'])]

chord_number = 2
print(lables_D[chord_number])
ipd.Audio(beatles[0][bounds_D[chord_number][0]:bounds_D[chord_number][1]], rate=beatles[1])

In [ ]:
add_G = np.zeros(len(beatles[0]))
for beginning, end in bounds_G:
    if end > len(beatles[0]):
        end = len(beatles[0])
    add = note_by_num_samples(98., end-beginning)
    add_G[beginning:end] = add
    
output = beatles[0] + add_G
ipd.Audio(output, rate=beatles[1])

In [ ]:
add_D = np.zeros(len(beatles[0]))
for beginning, end in bounds_D:
    if end > len(beatles[0]):
        end = len(beatles[0])
    add = note_by_num_samples(73.42 , end-beginning)
    add_D[beginning:end] = add
    
output = beatles[0] + add_D
ipd.Audio(output, rate=beatles[1])

In [ ]:
list(chords[:-1])

In [ ]:
last_chord_index = -1
while chords['list'][last_chord_index]['label'] == 'N':
    last_chord_index -= 1
last_chord = chords['list'][last_chord_index]['label']
print(last_chord)

In [ ]:
for chord in chords['list']:
    print(chord)

In [ ]:
numb_appearance_chord = {}
for chord in chords['list']:
    numb_appearance_chord.setdefault(chord['label'], 0)
    numb_appearance_chord[chord['label']] += 1
print(numb_appearance_chord)
most_appearing_chord = ('', 0)
#for chord in numb_appearance_chord:
    


In [ ]:
import collections
list_of_chords = [chord['label'] for chord in chords['list']]
numb_appearance_chord2 = collections.Counter(list_of_chords)
print(numb_appearance_chord2)
most_appearing_chord = numb_appearance_chord2.most_common(1)
print(most_appearing_chord)

In [ ]:
chromagram = librosa.feature.chroma_stft(beatles[0], sr=beatles[1], hop_length=512)
plt.figure(figsize=(15, 5))
chromagram = np.log10(1+10*abs(chromagram))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=512, cmap='coolwarm')

In [ ]:
X = librosa.stft(beatles[0], n_fft=1024, hop_length=512) #raw ft output (complex), n_fft is window-length, ft is with respect to n_fft

In [ ]:
librosa.display.specshow(X, x_axis='time', hop_length=512) #displaying spectrogram

In [ ]:
Xfilt = np.array([v if i < 25 else np.zeros(X.shape[1]) for i, v in enumerate(X)]) #filtering out frequencies below 270Hz

In [ ]:
Xfilt

In [ ]:
audio = librosa.istft(Xfilt, win_length=1024, hop_length=512)

In [ ]:
ipd.Audio(audio, rate = beatles[1])

In [ ]:
# Xdb = librosa.amplitude_to_db(abs(Xfilt)) #length of X after scaling
Xdb = abs(Xfilt)

In [ ]:
librosa.display.specshow(Xdb, x_axis='time', y_axis='log', hop_length=512) #display log spectrogram

In [ ]:
Xdb.shape

In [ ]:
librosa.fft_frequencies(n_fft=1024) #start frqeuency of bins in ft output for n_fft=1024

In [ ]:
chroma_filters = librosa.filters.chroma(beatles[1], 1024)

In [ ]:
bass_chromagram = np.array([np.dot(chroma_filters, frame) for frame in Xdb.transpose()]).transpose()

In [ ]:
librosa.display.specshow(bass_chromagram, x_axis='time', y_axis='chroma', hop_length=512, cmap='coolwarm')

In [ ]:
weights = [np.sum(bin) for bin in bass_chromagram]
weights /= np.max(weights)
print(weights)

In [ ]:
librosa.display.specshow(np.array([weights]).transpose(), x_axis='time', y_axis='chroma', hop_length=512, cmap='coolwarm')

In [ ]:
ipd.Audio(beatles[0], rate = beatles[1])

In [ ]:
chromagram[0]

In [ ]:
weights = [np.sum(bin) for bin in chromagram]
weights /= np.max(weights)
print(weights)

In [ ]:
librosa.display.specshow(np.array([weights]).transpose(), x_axis='time', y_axis='chroma', hop_length=512, cmap='coolwarm')

In [ ]:
librosa.core.stft(beatles[0], n_fft=len(beatles[0])-1, hop_length=len(beatles[0])-1)

In [ ]:
print(chroma)

In [ ]:
S=chromadata.transpose()
librosa.display.specshow(S, sr=beatles[1], hop_length=2056, x_axis='time')
plt.colorbar()
plt.figure(figsize=(5, 50))